Sentiment Analysis of Goodreads Literature Reviews Using a RoBERTa-base Model

Loading the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import zipfile
import os
import json
import gzip
from itertools import islice
import csv
import glob

Since the original gz file with review data is too big for github, it was separated into several smaller gz files that were then saved in chunks

In [2]:
# Get the directory of the current file
DIR = os.getcwd()


In [3]:
# A function to split the original gz file into smaller gzs (to upload to GitHub)
def split_gz(gz_file_name, output_files):
    # Read ing all lines from the gzipped file just so the end files are not sepaerated with errors
    with gzip.open(gz_file_name, 'rt') as fin:
        lines = fin.readlines()

    # Calculate the number of lines in the big file
    total_lines = len(lines)
    # Number of lines per file
    lines_per_file = total_lines // len(output_files)
    remainder = total_lines % len(output_files)

    # Writing lines to the output smaller gzipped files
    start = 0
    for i, output_file in enumerate(output_files):
        # The end index for the current chunk
        end = start + lines_per_file + (1 if i < remainder else 0)
        # Write the chunk to the output file
        with gzip.open(output_file, 'wt') as fout:
            fout.writelines(lines[start:end])
        # Update the start index for the next chunk
        start = end
        # Print the name of the saved file
        print(f'Saved {os.path.basename(output_file)}')

In [9]:
"""
Since the original reviews file is too big to upload to GitHub, we split it into smaller files.
This is the code that was used for splitting, the resulting files are uploaded to GitHub
# File name 
gz_file_name = os.path.join(DIR, 'data/goodreads_reviews.gz')

# Making a directory to save the split files
output_dir = os.path.join(DIR, 'data/split_files')
os.makedirs('data/split_files', exist_ok=True)

# Setting 7 output files to save the split data
gz_file_names = [os.path.join(output_dir, f'goodreads_reviews_part_{i+1}.gz') for i in range(7)]
"""

In [10]:
"""
Actual splitting of the file
# Splitting the original gz file into parts
split_gz(gz_file_name, gz_file_names)
"""

Saved goodreads_reviews_part_1.gz
Saved goodreads_reviews_part_2.gz
Saved goodreads_reviews_part_3.gz
Saved goodreads_reviews_part_4.gz
Saved goodreads_reviews_part_5.gz
Saved goodreads_reviews_part_6.gz
Saved goodreads_reviews_part_7.gz


The smaller gz files then are iterated over in chunks (to avoid memory issues) and saved as csvs (used later for the rest of the data sets)

There were lines with errors due to merging the csv after separating .gz into 4 parts, for which I couldnt find an actual solution. 
This was fixed by reading the original .gz line by line and making more splits.

In [4]:
# A function to read a gz file line by line and yield chunks
def load_data_in_chunks(gz_file_name, chunk_size=200000):
    chunk = []
    with gzip.open(gz_file_name, 'rt', encoding='utf-8') as fin:
        for line in fin:
            line = line.strip()
            if not line:
                continue
            try:
                data = json.loads(line)
                chunk.append(data)
                if len(chunk) >= chunk_size:
                    yield chunk
                    chunk = []
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {gz_file_name}: {e}")
                continue
    if chunk:
        yield chunk

# A function to save the chunk as a CSV
def save_chunk_as_csv(chunk, output_file):
    #Converting  the chunk to a data frame
    chunk_df = pd.DataFrame(chunk)
    
    #Converting dictionaries inside to json strings to preserve structure
    for col in chunk_df.columns:
        if any(isinstance(val, dict) for val in chunk_df[col]):
            chunk_df[col] = chunk_df[col].apply(json.dumps)
    
    # Saving the chunk as a csv file
    chunk_df.to_csv(output_file, index=False)

# A function to process multiple gzipped json files
def process_gz_files(gz_file_names, output_dir, chunk_size=200000):
    chunk_counter = 0
    for gz_file_name in gz_file_names:
        # Processing the file in chunks
        for chunk in load_data_in_chunks(gz_file_name, chunk_size):
            # Saving the chunk as a csv file with a generic name (no part)
            chunk_file_name = os.path.join(output_dir, f'goodreads_reviews_chunk_{chunk_counter}.csv')
            save_chunk_as_csv(chunk, chunk_file_name)
            chunk_counter += 1
        print(f'Finished processing {gz_file_name}')
    return chunk_counter

In [23]:
# Output directory for csv chunk files
output_dir = os.path.join(DIR, 'data/chunks')
os.makedirs(output_dir, exist_ok=True)

# the files - inside the split_files directory

small_gz_file_names = [os.path.join(DIR, 'data/split_files', f'goodreads_reviews_part_{i}.gz') for i in range(1, 8)
]

chunk_counter = process_gz_files(small_gz_file_names, output_dir)

Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_reviews_part_1.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_reviews_part_2.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_reviews_part_3.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_reviews_part_4.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_reviews_part_5.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_reviews_part_6.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_reviews_part_7.gz


In [5]:
# A function to merge the chunks into a single file to be used for analysis
# dir_path: a directory where the chunk files are saved
# file_pattern: a pattern for the chunk file names
# num_chunks: number of chunks

def merge_chunks_to_dataframe(dir_path, file_pattern, num_chunks):
    #Defining the file pattern for chunk files to merge them together later
    chunk_files = [os.path.join(dir_path, file_pattern.format(i)) for i in range(num_chunks)]
    # A list to store data frames
    dataframes = []
    # Storing the columns of the first data frame - makes sure all data frames have the same structure
    first_df_columns = None
    # Iterating over the chunk files
    for i, file in enumerate(chunk_files):
        # Reading the entire chunk file
        df = pd.read_csv(file, low_memory=False)
        # If it's the first file, store its columns
        if first_df_columns is None:
            first_df_columns = df.columns
        #Checking if the data frame has the same columns as the first data frame
        if not df.columns.equals(first_df_columns):
            raise ValueError(f"File {os.path.basename(file)} has a different structure")
        #Adding resulting df to the list
        dataframes.append(df)
        print(f'Processed file {os.path.basename(file)}')
    # Merging all dfs into a 1
    merged_df = pd.concat(dataframes, ignore_index=True)
    print('Merged DataFrame created')
    return merged_df


In [6]:
# Implementing merge chunks for the reviews

# The directory where the chunk files are saved and file pattern 
dir_path = os.path.join(DIR, 'data/chunks')
file_pattern = 'goodreads_reviews_chunk_{}.csv'

# Checking how many chunk files are there
chunk_pattern = os.path.join(dir_path, 'goodreads_reviews_chunk_*.csv')

# glob to find all files matching the pattern
matching = glob.glob(chunk_pattern)

# Count the number of matching files
chunk_counter = len(matching)

#Merging the chunks into a single data frame
reviews_df = merge_chunks_to_dataframe(dir_path, file_pattern, chunk_counter)

Processed file goodreads_reviews_chunk_0.csv
Processed file goodreads_reviews_chunk_1.csv
Processed file goodreads_reviews_chunk_2.csv
Processed file goodreads_reviews_chunk_3.csv
Processed file goodreads_reviews_chunk_4.csv
Processed file goodreads_reviews_chunk_5.csv
Processed file goodreads_reviews_chunk_6.csv
Processed file goodreads_reviews_chunk_7.csv
Processed file goodreads_reviews_chunk_8.csv
Processed file goodreads_reviews_chunk_9.csv
Processed file goodreads_reviews_chunk_10.csv
Processed file goodreads_reviews_chunk_11.csv
Processed file goodreads_reviews_chunk_12.csv
Processed file goodreads_reviews_chunk_13.csv
Processed file goodreads_reviews_chunk_14.csv
Processed file goodreads_reviews_chunk_15.csv
Processed file goodreads_reviews_chunk_16.csv
Processed file goodreads_reviews_chunk_17.csv
Processed file goodreads_reviews_chunk_18.csv
Processed file goodreads_reviews_chunk_19.csv
Processed file goodreads_reviews_chunk_20.csv
Processed file goodreads_reviews_chunk_21.cs

In [7]:
reviews_df

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,24375664,5cd416f3efc3f944fce4ce2db2290d5e,5,Mind blowingly cool. Best science fiction I've...,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,Sat Oct 07 00:00:00 -0700 2017,Sat Aug 26 00:00:00 -0700 2017,16,0
1,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
2,8842281e1d1347389f2ab93d60773d4d,6392944,5e212a62bced17b4dbe41150e5bb9037,3,I haven't read a fun mystery book in a while a...,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,Mon Jul 24 00:00:00 -0700 2017,6,0
3,8842281e1d1347389f2ab93d60773d4d,22078596,fdd13cad0695656be99828cd75d6eb73,4,"Fun, fast paced, and disturbing tale of murder...",Mon Jul 24 02:33:09 -0700 2017,Sun Jul 30 10:23:54 -0700 2017,Sun Jul 30 15:42:05 -0700 2017,Tue Jul 25 00:00:00 -0700 2017,22,4
4,8842281e1d1347389f2ab93d60773d4d,6644782,bd0df91c9d918c0e433b9ab3a9a5c451,4,A fun book that gives you a sense of living in...,Mon Jul 24 02:28:14 -0700 2017,Thu Aug 24 00:07:20 -0700 2017,Sat Aug 05 00:00:00 -0700 2017,Sun Jul 30 00:00:00 -0700 2017,8,0
...,...,...,...,...,...,...,...,...,...,...,...
15739962,d0f6d1a4edcab80a6010cfcfeda4999f,1656001,b3d9a00405f7e96752d67b85deda4c7d,4,"I started the first chapter, can't seem to get...",Mon Jun 04 18:08:44 -0700 2012,Tue Jun 26 18:58:46 -0700 2012,NaN,Sun Jun 10 00:00:00 -0700 2012,0,1
15739963,594c86711bd7acdaf655d102df52a9cb,10024429,2bcba3579aa1d728e664de293e16aacf,5,lmdh lm tdqWw jdrn lkhzWn? lmdh lm tqr`w jdrn ...,Fri Aug 01 18:46:18 -0700 2014,Fri Aug 01 18:47:07 -0700 2014,NaN,NaN,0,0
15739964,594c86711bd7acdaf655d102df52a9cb,6721437,7c1a7fcc2614a1a2a29213c11c991083,3,y`n~ n tfSlt mn km ltshtyt ll~ fyh.. w msh qdr...,Tue Aug 27 12:49:25 -0700 2013,Tue Aug 27 12:53:46 -0700 2013,NaN,NaN,0,0
15739965,594c86711bd7acdaf655d102df52a9cb,15788197,74a9f9d1db09a90aae3a5acea68c6593,2,nSh lwln~ mlwsh `lq@ bltn~ w mkhtlf tmm`nh lwl...,Fri May 03 13:06:15 -0700 2013,Fri May 03 15:35:39 -0700 2013,Fri May 03 15:35:39 -0700 2013,Fri May 03 00:00:00 -0700 2013,0,0


In [8]:
#Getting the columns needed only
columns = ['book_id', 'user_id', 'rating', 'review_text','n_votes']
reviews_df = reviews_df[columns]

In [9]:
reviews_df

,book_id,user_id,rating,review_text,n_votes
0,24375664,8842281e1d1347389f2ab93d60773d4d,5,Mind blowingly cool. Best science fiction I've...,16
1,18245960,8842281e1d1347389f2ab93d60773d4d,5,This is a special book. It started slow for ab...,28
2,6392944,8842281e1d1347389f2ab93d60773d4d,3,I haven't read a fun mystery book in a while a...,6
3,22078596,8842281e1d1347389f2ab93d60773d4d,4,"Fun, fast paced, and disturbing tale of murder...",22
4,6644782,8842281e1d1347389f2ab93d60773d4d,4,A fun book that gives you a sense of living in...,8
...,...,...,...,...,...
15739962,1656001,d0f6d1a4edcab80a6010cfcfeda4999f,4,"I started the first chapter, can't seem to get...",0
15739963,10024429,594c86711bd7acdaf655d102df52a9cb,5,lmdh lm tdqWw jdrn lkhzWn? lmdh lm tqr`w jdrn ...,0
15739964,6721437,594c86711bd7acdaf655d102df52a9cb,3,y`n~ n tfSlt mn km ltshtyt ll~ fyh.. w msh qdr...,0
15739965,15788197,594c86711bd7acdaf655d102df52a9cb,2,nSh lwln~ mlwsh `lq@ bltn~ w mkhtlf tmm`nh lwl...,0


Book data

In [29]:
# Splitting the original book gz file into parts

# File name 
gz_file_name = os.path.join(DIR, 'data/goodreads_books.gz')

# Output directory for the split files
output_dir = os.path.join(DIR, 'data/split_files')

# Setting 5 output files to save the split data
splits = 5
gz_file_names = [os.path.join(output_dir, f'goodreads_books_part_{i+1}.gz') for i in range(splits)]


In [30]:
# Splitting
split_gz(gz_file_name, gz_file_names)

Saved goodreads_books_part_1.gz
Saved goodreads_books_part_2.gz
Saved goodreads_books_part_3.gz
Saved goodreads_books_part_4.gz
Saved goodreads_books_part_5.gz


In [10]:
# A function to process multiple gzipped json files - changing the name of the output files
def process_gz_files(gz_file_names, output_dir, chunk_size=200000):
    chunk_counter = 0
    for gz_file_name in gz_file_names:
        # Processing the file in chunks
        for chunk in load_data_in_chunks(gz_file_name, chunk_size):
            # Saving the chunk as a csv file with a generic name (no part)
            chunk_file_name = os.path.join(output_dir, f'goodreads_books_chunk_{chunk_counter}.csv')
            save_chunk_as_csv(chunk, chunk_file_name)
            chunk_counter += 1
        print(f'Finished processing {gz_file_name}')
    return chunk_counter


In [34]:
# Splitting smaller gz files into chunks

output_dir = os.path.join(DIR, 'data/chunks')

# the files - inside the split_files directory

small_gz_file_names = [os.path.join(DIR, 'data/split_files', f'goodreads_books_part_{i+1}.gz') for i in range(splits)
]

chunk_counter = process_gz_files(small_gz_file_names, output_dir)

Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_books_part_1.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_books_part_2.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_books_part_3.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_books_part_4.gz
Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/split_files\goodreads_books_part_5.gz


In [10]:
# Implementing merge chunks for the reviews

# The directory where the chunk files are saved and file pattern 
dir_path = os.path.join(DIR, 'data/chunks')
file_pattern = 'goodreads_books_chunk_{}.csv'

# Checking how many chunk files are there
chunk_pattern = os.path.join(dir_path, 'goodreads_books_chunk_*.csv')

# glob to find all files matching the pattern
matching = glob.glob(chunk_pattern)

# Count the number of matching files
chunk_counter = len(matching)
#Merging the chunks into a single data frame
books_df = merge_chunks_to_dataframe(dir_path, file_pattern, chunk_counter)

Processed file goodreads_books_chunk_0.csv
Processed file goodreads_books_chunk_1.csv
Processed file goodreads_books_chunk_2.csv
Processed file goodreads_books_chunk_3.csv
Processed file goodreads_books_chunk_4.csv
Processed file goodreads_books_chunk_5.csv
Processed file goodreads_books_chunk_6.csv
Processed file goodreads_books_chunk_7.csv
Processed file goodreads_books_chunk_8.csv
Processed file goodreads_books_chunk_9.csv
Processed file goodreads_books_chunk_10.csv
Processed file goodreads_books_chunk_11.csv
Processed file goodreads_books_chunk_12.csv
Processed file goodreads_books_chunk_13.csv
Processed file goodreads_books_chunk_14.csv
Merged DataFrame created


In [11]:
books_df

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,0312853122,1.0,[],US,NaN,"[{'count': '3', 'name': 'to-read'}, {'count': ...",NaN,False,4.00,NaN,...,9.0,NaN,1984.0,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3.0,5400751.0,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film
1,0743509986,6.0,[],US,NaN,"[{'count': '2634', 'name': 'to-read'}, {'count...",NaN,False,3.23,B000FC0PBC,...,10.0,Abridged,2001.0,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10.0,1323437.0,Good Harbor,Good Harbor
2,NaN,7.0,['189911'],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,False,4.03,NaN,...,NaN,Book Club Edition,1987.0,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140.0,8948723.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3282.0,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",NaN,False,3.49,B002ENBLOK,...,7.0,NaN,2009.0,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184.0,6243154.0,Best Friends Forever,Best Friends Forever
4,0850308712,5.0,[],US,NaN,"[{'count': '32', 'name': 'to-read'}, {'count':...",NaN,False,3.40,NaN,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15.0,278577.0,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2360650,0563553014,3.0,['618749'],US,eng,"[{'count': '11', 'name': 'to-read'}, {'count':...",NaN,False,4.05,NaN,...,9.0,BBC Radio Collection,1999.0,https://www.goodreads.com/book/show/3084038-th...,https://images.gr-assets.com/books/1494763458m...,3084038,12.0,3115103.0,"This Sceptred Isle, Vol. 10: The Age of Victor...","This Sceptred Isle, Vol. 10: The Age of Victor..."
2360651,178092870X,2.0,[],US,eng,"[{'count': '702', 'name': 'to-read'}, {'count'...",NaN,False,3.50,NaN,...,8.0,2nd Edition,2015.0,https://www.goodreads.com/book/show/26168430-s...,https://images.gr-assets.com/books/1440592011m...,26168430,6.0,46130263.0,Sherlock Holmes and the July Crisis,Sherlock Holmes and the July Crisis
2360652,0765197456,6.0,[],US,NaN,"[{'count': '37', 'name': 'to-read'}, {'count':...",NaN,False,4.00,NaN,...,8.0,NaN,1996.0,https://www.goodreads.com/book/show/2342551.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,2342551,36.0,2349247.0,The Children's Classic Poetry Collection,The Children's Classic Poetry Collection
2360653,162378140X,17.0,['658195'],US,eng,"[{'count': '56', 'name': 'to-read'}, {'count':...",NaN,False,4.37,NaN,...,4.0,NaN,2014.0,https://www.goodreads.com/book/show/22017381-1...,https://images.gr-assets.com/books/1398621236m...,22017381,70.0,41332799.0,"101 Nights: Volume One (101 Nights, #1-3)","101 Nights: Volume One (101 Nights, #1-3)"


In [12]:
#Checking the coulmns of the books data
books_df.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')

Since the models that will be used are for English-lannguage content, we will filter out the books written in English only

In [13]:
# Checking which languages are the books written in 
books_df['language_code'].value_counts()

language_code
eng      708457
en-US     91452
en-GB     58358
spa       54524
ita       50902
          ...  
hat           1
ltz           1
btk           1
sla           1
lao           1
Name: count, Length: 226, dtype: int64

In [14]:
#Filtering the books  for English books
eng_books = books_df[books_df['language_code'].isin(['eng', 'en-US', 'en-GB'])]

In [15]:
eng_books

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
2,NaN,7.0,['189911'],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,False,4.03,NaN,...,NaN,Book Club Edition,1987.0,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140.0,8948723.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3282.0,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",NaN,False,3.49,B002ENBLOK,...,7.0,NaN,2009.0,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184.0,6243154.0,Best Friends Forever,Best Friends Forever
11,NaN,60.0,['1052227'],US,eng,"[{'count': '54', 'name': 'currently-reading'},...",B01NCIKAQX,True,4.33,B01NCIKAQX,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/33394837-t...,https://images.gr-assets.com/books/1493114742m...,33394837,269.0,54143148.0,The House of Memory (Pluto's Snitch #2),The House of Memory (Pluto's Snitch #2)
14,555118000X,19.0,[],US,eng,"[{'count': '3488', 'name': 'to-read'}, {'count...",NaN,False,3.82,NaN,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/89373.The_...,https://s.gr-assets.com/assets/nophoto/book/11...,89373,77.0,1080201.0,The Bonfire of the Vanities,The Bonfire of the Vanities
16,0842379428,566.0,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",NaN,False,4.26,B000FCKCJC,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/89376.Heaven,https://images.gr-assets.com/books/1406508230m...,89376,7345.0,86257.0,Heaven,Heaven
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2360643,NaN,27.0,['494165'],US,en-GB,"[{'count': '850', 'name': 'to-read'}, {'count'...",B00HETCFBW,True,4.02,B00HETCFBW,...,12.0,NaN,2013.0,https://www.goodreads.com/book/show/17805813-o...,https://images.gr-assets.com/books/1379766592m...,17805813,327.0,24908707.0,"Ondine (Ondine Quartet, #0.5)","Ondine (Ondine Quartet, #0.5)"
2360648,NaN,3.0,[],US,eng,"[{'count': '4', 'name': 'to-read'}, {'count': ...",NaN,True,2.00,NaN,...,5.0,NaN,2013.0,https://www.goodreads.com/book/show/18069148-d...,https://images.gr-assets.com/books/1370997860m...,18069148,2.0,25365816.0,Different Breeds,Different Breeds
2360650,0563553014,3.0,['618749'],US,eng,"[{'count': '11', 'name': 'to-read'}, {'count':...",NaN,False,4.05,NaN,...,9.0,BBC Radio Collection,1999.0,https://www.goodreads.com/book/show/3084038-th...,https://images.gr-assets.com/books/1494763458m...,3084038,12.0,3115103.0,"This Sceptred Isle, Vol. 10: The Age of Victor...","This Sceptred Isle, Vol. 10: The Age of Victor..."
2360651,178092870X,2.0,[],US,eng,"[{'count': '702', 'name': 'to-read'}, {'count'...",NaN,False,3.50,NaN,...,8.0,2nd Edition,2015.0,https://www.goodreads.com/book/show/26168430-s...,https://images.gr-assets.com/books/1440592011m...,26168430,6.0,46130263.0,Sherlock Holmes and the July Crisis,Sherlock Holmes and the July Crisis


In [16]:
# Getting only the columns that are relevant for the analysis
columns = ['book_id', 'title', 'popular_shelves', 'num_pages', 'publication_year', 'ratings_count', 'average_rating', 'text_reviews_count']
eng_books = eng_books[columns]

In [17]:
# There are some nan values in the data, leave for now since will be merging later
eng_books.isna().sum()

book_id                    0
title                      4
popular_shelves            0
num_pages             255722
publication_year      187897
ratings_count              6
average_rating             6
text_reviews_count         6
dtype: int64

In [18]:
eng_books

,book_id,title,popular_shelves,num_pages,publication_year,ratings_count,average_rating,text_reviews_count
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[{'count': '58', 'name': 'to-read'}, {'count':...",600.0,1987.0,140.0,4.03,7.0
3,6066819,Best Friends Forever,"[{'count': '7615', 'name': 'to-read'}, {'count...",368.0,2009.0,51184.0,3.49,3282.0
11,33394837,The House of Memory (Pluto's Snitch #2),"[{'count': '54', 'name': 'currently-reading'},...",318.0,NaN,269.0,4.33,60.0
14,89373,The Bonfire of the Vanities,"[{'count': '3488', 'name': 'to-read'}, {'count...",NaN,NaN,77.0,3.82,19.0
16,89376,Heaven,"[{'count': '6393', 'name': 'to-read'}, {'count...",533.0,NaN,7345.0,4.26,566.0
...,...,...,...,...,...,...,...,...
2360643,17805813,"Ondine (Ondine Quartet, #0.5)","[{'count': '850', 'name': 'to-read'}, {'count'...",84.0,2013.0,327.0,4.02,27.0
2360648,18069148,Different Breeds,"[{'count': '4', 'name': 'to-read'}, {'count': ...",NaN,2013.0,2.0,2.00,3.0
2360650,3084038,"This Sceptred Isle, Vol. 10: The Age of Victor...","[{'count': '11', 'name': 'to-read'}, {'count':...",3.0,1999.0,12.0,4.05,3.0
2360651,26168430,Sherlock Holmes and the July Crisis,"[{'count': '702', 'name': 'to-read'}, {'count'...",148.0,2015.0,6.0,3.50,2.0


Genre data - these are fuzzy book genres that were extracted by the authors of the dataset via users' popular shelves by keyword matching process

Since genres data set is small already, no need for splitting

In [19]:
# A function to process multiple gzipped json files - changing the name of the output files
def process_gz_files(gz_file_names, output_dir, chunk_size=200000):
    chunk_counter = 0
    for gz_file_name in gz_file_names:
        # Processing the file in chunks
        for chunk in load_data_in_chunks(gz_file_name, chunk_size):
            # Saving the chunk as a csv file with a generic name (no part)
            chunk_file_name = os.path.join(output_dir, f'goodreads_genres_chunk_{chunk_counter}.csv')
            save_chunk_as_csv(chunk, chunk_file_name)
            chunk_counter += 1
        print(f'Finished processing {gz_file_name}')
    return chunk_counter

In [53]:
# Output directory for csv chunk files
output_dir = os.path.join(DIR, 'data/chunks')

# the files - inside the split_files directory

small_gz_file_names = [os.path.join(DIR, 'data/goodreads_genres.gz')]

chunk_counter = process_gz_files(small_gz_file_names, output_dir)

Finished processing c:\Users\lemon\OneDrive\Documents\SMDAproject\data/goodreads_genres.gz


In [19]:
# Implementing merge chunks for the genres

# The directory where the chunk files are saved and file pattern 
dir_path = os.path.join(DIR, 'data/chunks')
file_pattern = 'goodreads_genres_chunk_{}.csv'

# Checking how many chunk files are there
chunk_pattern = os.path.join(dir_path, 'goodreads_genres_chunk_*.csv')

# glob to find all files matching the pattern
matching = glob.glob(chunk_pattern)

# Count the number of matching files
chunk_counter = len(matching)

#Merging the chunks into a single data frame
genres_df = merge_chunks_to_dataframe(dir_path, file_pattern, chunk_counter)

Processed file goodreads_genres_chunk_0.csv
Processed file goodreads_genres_chunk_1.csv
Processed file goodreads_genres_chunk_2.csv
Processed file goodreads_genres_chunk_3.csv
Processed file goodreads_genres_chunk_4.csv
Processed file goodreads_genres_chunk_5.csv
Processed file goodreads_genres_chunk_6.csv
Processed file goodreads_genres_chunk_7.csv
Processed file goodreads_genres_chunk_8.csv
Processed file goodreads_genres_chunk_9.csv
Processed file goodreads_genres_chunk_10.csv
Processed file goodreads_genres_chunk_11.csv
Merged DataFrame created


In [20]:
genres_df

,book_id,genres
0,5333265,"{""history, historical fiction, biography"": 1}"
1,1333909,"{""fiction"": 219, ""history, historical fiction,..."
2,7327624,"{""fantasy, paranormal"": 31, ""fiction"": 8, ""mys..."
3,6066819,"{""fiction"": 555, ""romance"": 23, ""mystery, thri..."
4,287140,"{""non-fiction"": 3}"
...,...,...
2360650,3084038,"{""non-fiction"": 5, ""history, historical fictio..."
2360651,26168430,"{""mystery, thriller, crime"": 4, ""children"": 1,..."
2360652,2342551,"{""poetry"": 14, ""children"": 7, ""young-adult"": 1..."
2360653,22017381,"{""romance"": 13, ""mystery, thriller, crime"": 2}"


Data Preprocessing

Merging the books and their genres on ids

In [21]:
engbooks_df = eng_books.merge(genres_df, on='book_id', how='left')

In [22]:
engbooks_df

,book_id,title,popular_shelves,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genres
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[{'count': '58', 'name': 'to-read'}, {'count':...",600.0,1987.0,140.0,4.03,7.0,"{""fantasy, paranormal"": 31, ""fiction"": 8, ""mys..."
1,6066819,Best Friends Forever,"[{'count': '7615', 'name': 'to-read'}, {'count...",368.0,2009.0,51184.0,3.49,3282.0,"{""fiction"": 555, ""romance"": 23, ""mystery, thri..."
2,33394837,The House of Memory (Pluto's Snitch #2),"[{'count': '54', 'name': 'currently-reading'},...",318.0,NaN,269.0,4.33,60.0,"{""fantasy, paranormal"": 11, ""mystery, thriller..."
3,89373,The Bonfire of the Vanities,"[{'count': '3488', 'name': 'to-read'}, {'count...",NaN,NaN,77.0,3.82,19.0,"{""fiction"": 1760, ""mystery, thriller, crime"": ..."
4,89376,Heaven,"[{'count': '6393', 'name': 'to-read'}, {'count...",533.0,NaN,7345.0,4.26,566.0,"{""non-fiction"": 163}"
...,...,...,...,...,...,...,...,...,...
858262,17805813,"Ondine (Ondine Quartet, #0.5)","[{'count': '850', 'name': 'to-read'}, {'count'...",84.0,2013.0,327.0,4.02,27.0,"{""young-adult"": 25, ""fantasy, paranormal"": 47,..."
858263,18069148,Different Breeds,"[{'count': '4', 'name': 'to-read'}, {'count': ...",NaN,2013.0,2.0,2.00,3.0,"{""romance"": 1}"
858264,3084038,"This Sceptred Isle, Vol. 10: The Age of Victor...","[{'count': '11', 'name': 'to-read'}, {'count':...",3.0,1999.0,12.0,4.05,3.0,"{""non-fiction"": 5, ""history, historical fictio..."
858265,26168430,Sherlock Holmes and the July Crisis,"[{'count': '702', 'name': 'to-read'}, {'count'...",148.0,2015.0,6.0,3.50,2.0,"{""mystery, thriller, crime"": 4, ""children"": 1,..."


In [23]:
# Checking for Nan values in the new df
engbooks_df.isna().sum()

book_id                    0
title                      4
popular_shelves            0
num_pages             255722
publication_year      187897
ratings_count              6
average_rating             6
text_reviews_count         6
genres                     0
dtype: int64

In [24]:
engbooks_df.dtypes

book_id                 int64
title                  object
popular_shelves        object
num_pages             float64
publication_year      float64
ratings_count         float64
average_rating        float64
text_reviews_count    float64
genres                 object
dtype: object

For the analysis of the review and wether they depend on the genres, we will need to extract the genre labels. For the data from the book data set (popular shelves), we will get the most frequent genre that was mentioned by the users.

For the genres that were extracted by the authors () of the data seta, we will do the same and then compare which categorization is better.

Since the size of the dataset is big and we want to save the proportion of most frequent genres in the book for the sampling later, we will get it by choosing the most frequent genre in the  dataframe with all of the data.

In [25]:
# Libraries for preprocessing
from collections import Counter
import ast

In [26]:
# A function to get the most frequent genres
def most_frequent_genre(genre_dict_str):
    # Ensure the input string is properly formatted as JSON
    if not isinstance(genre_dict_str, str):
        print(f"Invalid input: {genre_dict_str}")
        return None
    
    genre_dict_str = genre_dict_str.replace("'", '"')
    
    try:
        # Parsing the JSON string into a dictionary or list of dictionaries
        genre_data = json.loads(genre_dict_str)
    except (ValueError, TypeError) as e:
        print(f"JSON parsing error: {e} for input: {genre_dict_str}")
        return None
    
    # Initialize a Counter to count genres
    genre_counts = Counter()
    
    if isinstance(genre_data, list):
        # If the data is a list of dictionaries
        for item in genre_data:
            if 'name' in item and 'count' in item:
                genre_counts[item['name']] += int(item['count'])
    elif isinstance(genre_data, dict):
        # If the data is a dictionary
        for genre, count in genre_data.items():
            genre_counts[genre] += int(count)
    else:
        print(f"Unexpected data format: {genre_data}")
        return None
    
    most_freq = genre_counts.most_common(3)
    
    # Return up to 3 most frequent genres
    return [genre for genre, count in most_freq] if most_freq else None


In [27]:
# Applying the function to the popular_shelves column and saving into new columns
try:
    engbooks_df['most_frequent_popular_shelf'] = engbooks_df['popular_shelves'].apply(lambda x: most_frequent_genre(x)[0] if most_frequent_genre(x) else None)
    engbooks_df['second_most_frequent_popular_shelf'] = engbooks_df['popular_shelves'].apply(lambda x: most_frequent_genre(x)[1] if most_frequent_genre(x) and len(most_frequent_genre(x)) > 1 else None)
    engbooks_df['third_most_frequent_popular_shelf'] = engbooks_df['popular_shelves'].apply(lambda x: most_frequent_genre(x)[2] if most_frequent_genre(x) and len(most_frequent_genre(x)) > 2 else None)
except Exception as e:
    print(f"Error encountered: {e}")
    raise

In [28]:
engbooks_df

,book_id,title,popular_shelves,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genres,most_frequent_popular_shelf,second_most_frequent_popular_shelf,third_most_frequent_popular_shelf
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[{'count': '58', 'name': 'to-read'}, {'count':...",600.0,1987.0,140.0,4.03,7.0,"{""fantasy, paranormal"": 31, ""fiction"": 8, ""mys...",to-read,fantasy,fiction
1,6066819,Best Friends Forever,"[{'count': '7615', 'name': 'to-read'}, {'count...",368.0,2009.0,51184.0,3.49,3282.0,"{""fiction"": 555, ""romance"": 23, ""mystery, thri...",to-read,chick-lit,currently-reading
2,33394837,The House of Memory (Pluto's Snitch #2),"[{'count': '54', 'name': 'currently-reading'},...",318.0,NaN,269.0,4.33,60.0,"{""fantasy, paranormal"": 11, ""mystery, thriller...",currently-reading,netgalley,kindle
3,89373,The Bonfire of the Vanities,"[{'count': '3488', 'name': 'to-read'}, {'count...",NaN,NaN,77.0,3.82,19.0,"{""fiction"": 1760, ""mystery, thriller, crime"": ...",to-read,fiction,favorites
4,89376,Heaven,"[{'count': '6393', 'name': 'to-read'}, {'count...",533.0,NaN,7345.0,4.26,566.0,"{""non-fiction"": 163}",to-read,currently-reading,theology
...,...,...,...,...,...,...,...,...,...,...,...,...
858262,17805813,"Ondine (Ondine Quartet, #0.5)","[{'count': '850', 'name': 'to-read'}, {'count'...",84.0,2013.0,327.0,4.02,27.0,"{""young-adult"": 25, ""fantasy, paranormal"": 47,...",to-read,young-adult,paranormal
858263,18069148,Different Breeds,"[{'count': '4', 'name': 'to-read'}, {'count': ...",NaN,2013.0,2.0,2.00,3.0,"{""romance"": 1}",to-read,no-thanks,top2bottom-reviews
858264,3084038,"This Sceptred Isle, Vol. 10: The Age of Victor...","[{'count': '11', 'name': 'to-read'}, {'count':...",3.0,1999.0,12.0,4.05,3.0,"{""non-fiction"": 5, ""history, historical fictio...",to-read,non-fiction,audiobooks
858265,26168430,Sherlock Holmes and the July Crisis,"[{'count': '702', 'name': 'to-read'}, {'count'...",148.0,2015.0,6.0,3.50,2.0,"{""mystery, thriller, crime"": 4, ""children"": 1,...",to-read,mystery,giveaways


In [29]:
# Applying the function to the genres column and extracting only the most frequent genre
try:
    engbooks_df['most_frequent_genre'] = engbooks_df['genres'].apply(lambda x: most_frequent_genre(x)[0] if most_frequent_genre(x) else None)
    engbooks_df['second_most_frequent_genre'] = engbooks_df['genres'].apply(lambda x: most_frequent_genre(x)[1] if most_frequent_genre(x) and len(most_frequent_genre(x)) > 1 else None)
except Exception as e:
    print(f"Error encountered: {e}")
    raise

In [30]:
engbooks_df

,book_id,title,popular_shelves,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genres,most_frequent_popular_shelf,second_most_frequent_popular_shelf,third_most_frequent_popular_shelf,most_frequent_genre,second_most_frequent_genre
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[{'count': '58', 'name': 'to-read'}, {'count':...",600.0,1987.0,140.0,4.03,7.0,"{""fantasy, paranormal"": 31, ""fiction"": 8, ""mys...",to-read,fantasy,fiction,"fantasy, paranormal",fiction
1,6066819,Best Friends Forever,"[{'count': '7615', 'name': 'to-read'}, {'count...",368.0,2009.0,51184.0,3.49,3282.0,"{""fiction"": 555, ""romance"": 23, ""mystery, thri...",to-read,chick-lit,currently-reading,fiction,romance
2,33394837,The House of Memory (Pluto's Snitch #2),"[{'count': '54', 'name': 'currently-reading'},...",318.0,NaN,269.0,4.33,60.0,"{""fantasy, paranormal"": 11, ""mystery, thriller...",currently-reading,netgalley,kindle,"mystery, thriller, crime","fantasy, paranormal"
3,89373,The Bonfire of the Vanities,"[{'count': '3488', 'name': 'to-read'}, {'count...",NaN,NaN,77.0,3.82,19.0,"{""fiction"": 1760, ""mystery, thriller, crime"": ...",to-read,fiction,favorites,fiction,"mystery, thriller, crime"
4,89376,Heaven,"[{'count': '6393', 'name': 'to-read'}, {'count...",533.0,NaN,7345.0,4.26,566.0,"{""non-fiction"": 163}",to-read,currently-reading,theology,non-fiction,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858262,17805813,"Ondine (Ondine Quartet, #0.5)","[{'count': '850', 'name': 'to-read'}, {'count'...",84.0,2013.0,327.0,4.02,27.0,"{""young-adult"": 25, ""fantasy, paranormal"": 47,...",to-read,young-adult,paranormal,"fantasy, paranormal",young-adult
858263,18069148,Different Breeds,"[{'count': '4', 'name': 'to-read'}, {'count': ...",NaN,2013.0,2.0,2.00,3.0,"{""romance"": 1}",to-read,no-thanks,top2bottom-reviews,romance,None
858264,3084038,"This Sceptred Isle, Vol. 10: The Age of Victor...","[{'count': '11', 'name': 'to-read'}, {'count':...",3.0,1999.0,12.0,4.05,3.0,"{""non-fiction"": 5, ""history, historical fictio...",to-read,non-fiction,audiobooks,"history, historical fiction, biography",non-fiction
858265,26168430,Sherlock Holmes and the July Crisis,"[{'count': '702', 'name': 'to-read'}, {'count'...",148.0,2015.0,6.0,3.50,2.0,"{""mystery, thriller, crime"": 4, ""children"": 1,...",to-read,mystery,giveaways,"mystery, thriller, crime",children


In [31]:
engbooks_df.isna().sum()

book_id                                    0
title                                      4
popular_shelves                            0
num_pages                             255722
publication_year                      187897
ratings_count                              6
average_rating                             6
text_reviews_count                         6
genres                                     0
most_frequent_popular_shelf             7893
second_most_frequent_popular_shelf     25987
third_most_frequent_popular_shelf      43504
most_frequent_genre                    92925
second_most_frequent_genre            231902
dtype: int64

Based on the nan values, the most information dense columns are most_frequent_popular_shelf             (7893)
second_most_frequent_popular_shelf     (25987) and
most_frequent_genre                    (92925)


In [32]:
# Most frequent genre in the most_frequent_popular_shelf
engbooks_df['most_frequent_popular_shelf'].value_counts()

most_frequent_popular_shelf
to-read                            775863
currently-reading                   20563
fiction                              3229
classics                             2835
manga                                2786
                                    ...  
they-made-me-wot-i-am-shelf             1
i-m-sorry                               1
child-development-and-education         1
empty-book-shelf-challenge              1
useful-when-teaching                    1
Name: count, Length: 6883, dtype: int64

In [33]:
# Get second_most_frequent_popular_shelf
value_counts = engbooks_df['second_most_frequent_popular_shelf'].value_counts()

# Filter out the values that have a count of 900 or more
filtered_value_counts = value_counts.head(10)

filtered_value_counts

second_most_frequent_popular_shelf
currently-reading    229204
fantasy               37865
fiction               31571
to-read               30890
romance               29122
mystery               27472
m-m                   20809
young-adult           16403
kindle                15870
non-fiction           15795
Name: count, dtype: int64

However, as we can see, the genres from most popular shelves are mainly currently-reading or to-read labels, making them less suitable

In [34]:
# Get value counts for the most frequent genre
engbooks_df['most_frequent_genre'].value_counts()


most_frequent_genre
fiction                                   160335
romance                                   154425
fantasy, paranormal                       126337
mystery, thriller, crime                   87309
non-fiction                                80502
history, historical fiction, biography     51803
children                                   35012
comics, graphic                            33590
young-adult                                24869
poetry                                     11160
Name: count, dtype: int64

The most frequent genre is the best choice for the genre label

In [35]:
# Dropping the columns that are not needed
columns_to_drop = [
    'genres', 
    'most_frequent_popular_shelf', 
    'second_most_frequent_popular_shelf', 
    'third_most_frequent_popular_shelf', 
    'second_most_frequent_genre', 
    'popular_shelves'
]
engbooks_df.drop(columns=columns_to_drop, inplace=True)


In [36]:
# Renaming the 'most_frequent_genre' column to 'genre'
engbooks_df.rename(columns={'most_frequent_genre': 'genre'}, inplace=True)
engbooks_df


,book_id,title,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genre
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",600.0,1987.0,140.0,4.03,7.0,"fantasy, paranormal"
1,6066819,Best Friends Forever,368.0,2009.0,51184.0,3.49,3282.0,fiction
2,33394837,The House of Memory (Pluto's Snitch #2),318.0,NaN,269.0,4.33,60.0,"mystery, thriller, crime"
3,89373,The Bonfire of the Vanities,NaN,NaN,77.0,3.82,19.0,fiction
4,89376,Heaven,533.0,NaN,7345.0,4.26,566.0,non-fiction
...,...,...,...,...,...,...,...,...
858262,17805813,"Ondine (Ondine Quartet, #0.5)",84.0,2013.0,327.0,4.02,27.0,"fantasy, paranormal"
858263,18069148,Different Breeds,NaN,2013.0,2.0,2.00,3.0,romance
858264,3084038,"This Sceptred Isle, Vol. 10: The Age of Victor...",3.0,1999.0,12.0,4.05,3.0,"history, historical fiction, biography"
858265,26168430,Sherlock Holmes and the July Crisis,148.0,2015.0,6.0,3.50,2.0,"mystery, thriller, crime"


Merging the data with our reviews

In [37]:
reviews_books = reviews_df.merge(engbooks_df, on='book_id', how='left')
reviews_books.head()

,book_id,user_id,rating,review_text,n_votes,title,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genre
0,24375664,8842281e1d1347389f2ab93d60773d4d,5,Mind blowingly cool. Best science fiction I've...,16,"The Dark Forest (Remembrance of Earth’s Past, #2)",513.0,NaN,2925.0,4.38,154.0,fiction
1,18245960,8842281e1d1347389f2ab93d60773d4d,5,This is a special book. It started slow for ab...,28,The Three-Body Problem (Remembrance of Earth’s...,400.0,2014.0,6336.0,4.01,374.0,fiction
2,6392944,8842281e1d1347389f2ab93d60773d4d,3,I haven't read a fun mystery book in a while a...,6,"The Murder on the Links (Hercule Poirot, #2)",272.0,NaN,675.0,3.80,42.0,"mystery, thriller, crime"
3,22078596,8842281e1d1347389f2ab93d60773d4d,4,"Fun, fast paced, and disturbing tale of murder...",22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6644782,8842281e1d1347389f2ab93d60773d4d,4,A fun book that gives you a sense of living in...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Our data looks like this:

In [38]:
reviews_books

,book_id,user_id,rating,review_text,n_votes,title,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genre
0,24375664,8842281e1d1347389f2ab93d60773d4d,5,Mind blowingly cool. Best science fiction I've...,16,"The Dark Forest (Remembrance of Earth’s Past, #2)",513.0,NaN,2925.0,4.38,154.0,fiction
1,18245960,8842281e1d1347389f2ab93d60773d4d,5,This is a special book. It started slow for ab...,28,The Three-Body Problem (Remembrance of Earth’s...,400.0,2014.0,6336.0,4.01,374.0,fiction
2,6392944,8842281e1d1347389f2ab93d60773d4d,3,I haven't read a fun mystery book in a while a...,6,"The Murder on the Links (Hercule Poirot, #2)",272.0,NaN,675.0,3.80,42.0,"mystery, thriller, crime"
3,22078596,8842281e1d1347389f2ab93d60773d4d,4,"Fun, fast paced, and disturbing tale of murder...",22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6644782,8842281e1d1347389f2ab93d60773d4d,4,A fun book that gives you a sense of living in...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15739962,1656001,d0f6d1a4edcab80a6010cfcfeda4999f,4,"I started the first chapter, can't seem to get...",0,"The Host (The Host, #1)",620.0,2008.0,756375.0,3.84,36648.0,fiction
15739963,10024429,594c86711bd7acdaf655d102df52a9cb,5,lmdh lm tdqWw jdrn lkhzWn? lmdh lm tqr`w jdrn ...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15739964,6721437,594c86711bd7acdaf655d102df52a9cb,3,y`n~ n tfSlt mn km ltshtyt ll~ fyh.. w msh qdr...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15739965,15788197,594c86711bd7acdaf655d102df52a9cb,2,nSh lwln~ mlwsh `lq@ bltn~ w mkhtlf tmm`nh lwl...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Drop rows with missing genre values
reviews_books.dropna(subset=['genre'], inplace=True)

In [40]:
reviews_books

,book_id,user_id,rating,review_text,n_votes,title,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genre
0,24375664,8842281e1d1347389f2ab93d60773d4d,5,Mind blowingly cool. Best science fiction I've...,16,"The Dark Forest (Remembrance of Earth’s Past, #2)",513.0,NaN,2925.0,4.38,154.0,fiction
1,18245960,8842281e1d1347389f2ab93d60773d4d,5,This is a special book. It started slow for ab...,28,The Three-Body Problem (Remembrance of Earth’s...,400.0,2014.0,6336.0,4.01,374.0,fiction
2,6392944,8842281e1d1347389f2ab93d60773d4d,3,I haven't read a fun mystery book in a while a...,6,"The Murder on the Links (Hercule Poirot, #2)",272.0,NaN,675.0,3.80,42.0,"mystery, thriller, crime"
9,28114110,8842281e1d1347389f2ab93d60773d4d,4,"Kevin Kelly, who is a Wired co-founder, lays o...",14,The Inevitable: Understanding the 12 Technolog...,NaN,NaN,880.0,4.00,60.0,non-fiction
10,16981,8842281e1d1347389f2ab93d60773d4d,3,Recommended by Don Katz. Avail for free in Dec...,1,Invisible Man,581.0,1995.0,125232.0,3.84,3741.0,fiction
...,...,...,...,...,...,...,...,...,...,...,...,...
15739893,14740456,fc5aea6994fc754daefa27cbea10ac79,4,I generally liked the book. It had some parts ...,0,"The Prey (The Hunt, #2)",326.0,2013.0,3441.0,4.06,432.0,young-adult
15739894,7137327,fc5aea6994fc754daefa27cbea10ac79,3,To be honest I would probably rank it some whe...,0,"Enclave (Razorland, #1)",259.0,2011.0,55903.0,3.92,4378.0,young-adult
15739960,3412,d0f6d1a4edcab80a6010cfcfeda4999f,5,Truly a remarkable read......Meggie is your do...,0,The Thorn Birds,692.0,2003.0,258938.0,4.22,3736.0,fiction
15739961,2657,d0f6d1a4edcab80a6010cfcfeda4999f,5,An wonderfully written story of prejudice and ...,0,To Kill a Mockingbird,324.0,2006.0,3255518.0,4.26,59827.0,fiction


Getting genre stratification and then sampling accroding to the proportions

In [41]:
import math
import concurrent.futures

In [42]:
# Genre stratification
genre_counts = reviews_books.groupby('genre').size().reset_index(name='count')
total_books = genre_counts['count'].sum()
print(f"Total books: {total_books}")
genre_counts['proportion_books'] = (genre_counts['count'] / total_books).round(2)
genre_counts


# Sample size with a 95% confidence level
confidence_level = 0.95
# Z-score for 95% confidence
z_score = 1.96
# 5% margin of error
margin_of_error = 0.05
p = 0.5

# Calculating the sample size
numerator = (z_score ** 2) * p * (1 - p)
denominator = margin_of_error ** 2
sample_size = numerator / denominator

# Rounding the sample size so that multiplying is not affected by the tails
sample_size = round(sample_size)
print(f"Sample size: {sample_size}")

# Rounding the proportions to 2 decimal places so that multiplying is not affected by the tails
genre_counts['proportion_books'] = genre_counts['proportion_books'].round(2)

# Sample size for each genre
genre_counts['sample_size'] = (genre_counts['proportion_books'] * sample_size).round().astype(int)

# Display the genre stratification with sample sizes
genre_sample = genre_counts['sample_size'].sum()
print(f"Total sample size: {genre_sample}")
genre_ordered = genre_counts.sort_values(by = 'count', ascending=False)
genre_ordered

Total books: 11268781
Sample size: 384
Total sample size: 385


,genre,count,proportion_books,sample_size
3,fiction,2668725,0.24,92
2,"fantasy, paranormal",2450347,0.22,84
8,romance,2021444,0.18,69
5,"mystery, thriller, crime",1103909,0.10,38
9,young-adult,1058299,0.09,35
6,non-fiction,829938,0.07,27
1,"comics, graphic",377216,0.03,12
4,"history, historical fiction, biography",372806,0.03,12
0,children,328000,0.03,12
7,poetry,58097,0.01,4


In [43]:
list = genre_ordered['proportion_books'].to_list()
list

[0.24, 0.22, 0.18, 0.1, 0.09, 0.07, 0.03, 0.03, 0.03, 0.01]

In [44]:
# Performing stratified sampling

# A function to sample a subset of the data for each genre
def sample_iteration(df, genre_counts, sample_size_per_iteration, random_seed):
    # Empty data frame to store
    sampled_df = pd.DataFrame()
    
    # Sampling each genre
    for _, row in genre_counts.iterrows():
        # Genre and sample size
        genre = row['genre']
        # The sample size for the current genre based on proportions
        genre_sample_size = math.ceil(sample_size_per_iteration * (row['sample_size'] / genre_counts['sample_size'].sum()))
        
        # Sample Ids of books for the current genre
        sampled_books = df[df['genre'] == genre]['book_id'].drop_duplicates().sample(n=genre_sample_size, random_state=random_seed)
        
        # All reviews for the sampled books
        genre_df = df[df['book_id'].isin(sampled_books)]
        print(f"Number of reviews for sampled books in genre {genre}: {len(genre_df)}")
        
        # Append the sampled data to the data frame
        sampled_df = pd.concat([sampled_df, genre_df])
    
    return sampled_df

# A function to perform random stratified sampling in parallel
def random_stratified_sampling(df, genre_counts, total_sample_size, random_seed=None):
    # Sample size per iteration
    sample_size_per_iteration = 13000
    
    # List for the sampled data
    sampled_dfs = []
    total_rows_sampled = 0
    
    # Running sampling iterations in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while total_rows_sampled < total_sample_size:
            futures = [executor.submit(sample_iteration, df, genre_counts, sample_size_per_iteration, random_seed)]
            for future in concurrent.futures.as_completed(futures):
                # Getting the result of the completed future
                result_df = future.result()
                # And appending it to list of sampled dfs
                sampled_dfs.append(result_df)
                # Updating N of sampled so far
                total_rows_sampled += len(result_df)
                print(f"Total rows sampled so far: {total_rows_sampled}")
                # Break if total number of rows sampled reaches or exceeds 
                if total_rows_sampled >= total_sample_size:
                    break
    
    # Merging all sampled dfs into a single df and removing duplicates
    fin_sampled_df = pd.concat(sampled_dfs, ignore_index=True).drop_duplicates()
    
    # The final df has the needed number of rows
    if len(fin_sampled_df) > total_sample_size:
        fin_sampled_df = fin_sampled_df.sample(n=total_sample_size, random_state=random_seed)
    
    return fin_sampled_df


In [45]:
# Sampling

total_sample_size = 200000
random_seed = 49
fin_df = random_stratified_sampling(reviews_books, genre_counts, total_sample_size, random_seed)

Number of reviews for sampled books in genre children: 4433
Number of reviews for sampled books in genre comics, graphic: 5989
Number of reviews for sampled books in genre fantasy, paranormal: 46621
Number of reviews for sampled books in genre fiction: 52949
Number of reviews for sampled books in genre history, historical fiction, biography: 3263
Number of reviews for sampled books in genre mystery, thriller, crime: 14404
Number of reviews for sampled books in genre non-fiction: 9736
Number of reviews for sampled books in genre poetry: 805
Number of reviews for sampled books in genre romance: 31055
Number of reviews for sampled books in genre young-adult: 52735
Total rows sampled so far: 221990


In [46]:
fin_df

,book_id,user_id,rating,review_text,n_votes,title,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genre
214528,15790873,0b988f7755c59510cd6e913f29655277,4,"good book, cute characters, :)",0,This Is What Happy Looks Like (This is What Ha...,404.0,2013.0,40926.0,3.69,4275.0,young-adult
195979,15790873,6ca4829b5f4fadd1243ae569395f229b,0,DNF at page 137. \n Hated all the characters e...,0,This Is What Happy Looks Like (This is What Ha...,404.0,2013.0,40926.0,3.69,4275.0,young-adult
51279,7624272,11629342be6b91b1d65808ca3bae2d5a,5,Welcome back to the world of heroes and olympi...,0,"The Lost Hero (The Heroes of Olympus, #1)",17.0,2010.0,199075.0,4.35,8156.0,"fantasy, paranormal"
195486,23310761,08013927cc9f45094016b6f206671e76,3,I kind of jumped into this without knowing muc...,1,Tonight the Streets Are Ours,342.0,2015.0,2785.0,3.47,615.0,young-adult
58184,24111210,c309dff1695ed8558b29ea8dcd7479b8,5,"""We are not quite novels. \n We are not quite ...",1,The Storied Life of A.J. Fikry,258.0,2014.0,418.0,3.98,145.0,fiction
...,...,...,...,...,...,...,...,...,...,...,...,...
38189,12432220,ce8bc9247c32146d3161dcd8a7e2beaf,5,I absolutely love it when I can get completely...,3,The False Prince (The Ascendance Trilogy #1),342.0,2012.0,51292.0,4.23,6661.0,"fantasy, paranormal"
201491,25756328,45c6cd914eedbe668736c2a33d0f6a1a,4,"Una novela muy dulce, con unos personajes entr...",2,Love & Gelato,389.0,2016.0,13557.0,4.09,2596.0,young-adult
175665,6936382,dedc5ab26968ab14061c51432ba764a9,4,I read a lot of YA Book Blogs. I mean a lot. I...,0,Anna and the French Kiss (Anna and the French ...,372.0,2010.0,267364.0,4.07,20077.0,young-adult
65712,13604611,36ede39421232ff907111f926a28a8da,5,This book Is beautifully balanced between humo...,0,The Revised Fundamentals of Caregiving,290.0,2012.0,6993.0,3.77,1069.0,fiction


In [47]:
fin_df['book_id'].value_counts()


book_id
17333223    4892
6936382     4774
23395680    3850
6482837     3075
15790842    2755
            ... 
34739095       1
22728655       1
23558735       1
18226106       1
25295830       1
Name: count, Length: 12546, dtype: int64

We will need to use langdetect since not all english-language books have english-language reviews

In [48]:
pip install langdetect

In [49]:
# Removing non-English reviews with langdetect
from langdetect import detect, DetectorFactory
from concurrent.futures import ThreadPoolExecutor
#Setting seed for reproducibility
DetectorFactory.seed = 0

# A function to detect language
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# A function to filter non-English reviews
def filter_non_english_reviews(df, num_workers=4):
    # Splitting the df into smaller chunks for parallel processing
    chunks = np.array_split(df, num_workers)
    
    # Processing each chunk
    def process_chunk(chunk):
        chunk['is_english'] = chunk['review_text'].apply(is_english)
        return chunk[chunk['is_english']].drop(columns=['is_english'])
    
    #  Processing chunks in parallel
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(process_chunk, chunks)
    
    # Concatenate the results
    return pd.concat(results)

In [50]:
# Applying the function to filter non-English reviews
final_df = filter_non_english_reviews(fin_df)


c:\Users\lemon\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [51]:
final_df

,book_id,user_id,rating,review_text,n_votes,title,num_pages,publication_year,ratings_count,average_rating,text_reviews_count,genre
214528,15790873,0b988f7755c59510cd6e913f29655277,4,"good book, cute characters, :)",0,This Is What Happy Looks Like (This is What Ha...,404.0,2013.0,40926.0,3.69,4275.0,young-adult
195979,15790873,6ca4829b5f4fadd1243ae569395f229b,0,DNF at page 137. \n Hated all the characters e...,0,This Is What Happy Looks Like (This is What Ha...,404.0,2013.0,40926.0,3.69,4275.0,young-adult
51279,7624272,11629342be6b91b1d65808ca3bae2d5a,5,Welcome back to the world of heroes and olympi...,0,"The Lost Hero (The Heroes of Olympus, #1)",17.0,2010.0,199075.0,4.35,8156.0,"fantasy, paranormal"
195486,23310761,08013927cc9f45094016b6f206671e76,3,I kind of jumped into this without knowing muc...,1,Tonight the Streets Are Ours,342.0,2015.0,2785.0,3.47,615.0,young-adult
58184,24111210,c309dff1695ed8558b29ea8dcd7479b8,5,"""We are not quite novels. \n We are not quite ...",1,The Storied Life of A.J. Fikry,258.0,2014.0,418.0,3.98,145.0,fiction
...,...,...,...,...,...,...,...,...,...,...,...,...
107021,3367956,25f4d14d79e17ce94bb78a947f7b5336,5,I love this book--very well written. A real ey...,0,Hotel on the Corner of Bitter and Sweet,290.0,2009.0,201045.0,3.99,16484.0,fiction
101083,23206263,1b60ea7b5ac17ba5e4a4b083fed89260,2,I finished the book but I'm wondering if the a...,0,Firespill,NaN,NaN,8.0,3.26,2.0,fiction
40177,7825557,c1411eca9c6302bb4c144e2e526def0e,3,This was a fast read where Hansel and Gretel t...,0,"A Tale Dark & Grimm (A Tale Dark & Grimm, #1)",256.0,2010.0,15549.0,4.04,2432.0,"fantasy, paranormal"
134893,12143200,34bd09f66426d3f825edf61bb3ff938a,5,"Insightful, well documented, and terrifying.",6,Drift: The Unmooring of American Military Power,288.0,2012.0,13398.0,4.06,1645.0,non-fiction


In [52]:
# Saving the final data frame to a CSV file

final_df.to_csv('data/final_data.csv', index=False)


The analysis is conducted in the analysis.ipynb